In [16]:
%env VARIABLE=comptools-env

env: VARIABLE=comptools-env


In [17]:
import pandas as pd
books_data = pd.read_csv('data/books_data.csv') #, nrows=1000)
print(f"Books data has " + str(books_data.columns.size) + " columns with " + str(books_data.shape[0]) + " rows each.")
books_ratings = pd.read_csv('data/books_rating.csv') #, nrows=1000)
print(f"Books ratings has " + str(books_ratings.columns.size) + " columns with " + str(books_ratings.shape[0]) + " rows each.")

Books data has 10 columns with 212404 rows each.
Books ratings has 10 columns with 3000000 rows each.


In [18]:
# Titles
books_data_1 = books_data.drop(['image', 'previewLink', 'infoLink', 'publisher', 'ratingsCount'], axis=1, inplace=False)
books_ratings_1 = books_ratings.drop(['Id','Price', 'profileName', 'review/time'], axis=1, inplace=False)

books_data_1.rename(columns={'Title': 'title','publishedDate': 'published_date'}, inplace=True)
books_ratings_1.rename(columns={'Title': 'title','User_id':'user_id',  'review/score': 'score','review/helpfulness': 'helpfulness', 'review/text': 'text', 'review/summary': 'summary'}, inplace=True)
print(f"Books data now only has " + str(books_data_1.columns.size) + " columns - " + str(list(books_data_1.columns)))
print(f"Books ratings now only has " + str(books_ratings_1.columns.size) + " columns - " + str(list(books_ratings_1.columns))) 


Books data now only has 5 columns - ['title', 'description', 'authors', 'published_date', 'categories']
Books ratings now only has 6 columns - ['title', 'user_id', 'helpfulness', 'score', 'summary', 'text']


In [19]:
#Makes all titles lowercase; then removes any part of the title that includes a parenthesis, and replaces any ampersands with 'and'
books_data_1['title'] = books_data_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_data_1['title'] = books_data_1['title'].replace(r"\(.*\)","", regex=True)
books_data_1['title'] = books_data_1['title'].replace(r"\&","and", regex=True)

books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\(.*\)","", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\&","and", regex=True)

#Removes any characters that aren't letters or spaces.
books_data_1['title'] = books_data_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)

print("Both datasets' titles are now all lowercase and only include letters and spaces.")


Both datasets' titles are now all lowercase and only include letters and spaces.


In [20]:
# keep only unique titles in books_data_1
#size of books_data_1
prev_size = len(books_data_1['title'])
print(f"Books data has " + str(len(books_data_1['title'])) + " titles.")
books_data_1 = books_data_1.drop_duplicates(subset=['title'], keep='first')
print(f"Dropped " + str(prev_size - len(books_data_1['title'])) + " duplicate titles. Now there are only " + str(len(books_data_1['title'])) + " rows left. ")

#Remove NAN titles
print(f"" + str(books_ratings_1['title'].isnull().sum().sum()) + " total NAN values present in title")
books_ratings_1 = books_ratings_1.dropna(subset=['title'])
print(f"Now there are " + str(books_ratings_1['title'].isnull().sum().sum()))



Books data has 212404 titles.
Dropped 7500 duplicate titles. Now there are only 204904 rows left. 
208 total NAN values present in title
Now there are 0


In [21]:
#Drop duplicate reviews where same user reviews same book multiple times
print(f"Books ratings has " + str(len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(keep='first', inplace=True)
print(f"Dropped based on total duplicity. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(subset=['user_id', 'title'], keep='first', inplace=True)
print(f"Dropped based on duplicate title. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(subset=['user_id', 'text'], keep='first', inplace=True)
print(f"Dropped based on duplicate text. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")

#Drop duplicate books where the author and book description are equal.

prevBooks = len(books_data_1['title'])
books_data_1.drop_duplicates(subset=['description','authors'], keep='first', inplace=True)
print(f"Books data has " + str(books_data_1.shape[0]) + " rows." + " -" + str(prevBooks-len(books_data_1['title'])) + " rows.")


Books ratings has 2999792 rows.
Dropped based on total duplicity. Books ratings now has 2431827 rows.  -567965 rows.
Dropped based on duplicate title. Books ratings now has 2027122 rows.  -404705 rows.
Dropped based on duplicate text. Books ratings now has 1748097 rows.  -279025 rows.
Books data has 163712 rows. -41192 rows.


In [22]:
#Collect only ratings for books that have received at least 10 ratings.
#Add a new column in books_data_1 that counts the number of ratings for each book with a score of at least 10

#Note that by removing duplicate reviews first, counting the number of reviews for each book afterward will be more accurate.
print(f"Books_ratings has " + str(len(books_ratings_1['title'])) + " rows.")
prev_rows = len(books_ratings_1['title'])

n_ratings = books_ratings_1.groupby('title').count()['score']>=10
famous_books = n_ratings[n_ratings==True].index
books_ratings_1 = books_ratings_1[books_ratings_1['title'].isin(famous_books)]
n_ratings_1 = books_ratings_1['title'].value_counts().rename("n_ratings")
books_data_1 = books_data_1.merge(n_ratings_1, on='title', how='inner')

print(f"Books_ratings now has " + str(len(books_ratings_1['title'])) + " rows. -" + str(prev_rows - len(books_ratings_1['title'])) + " rows. ")
print(f"Books_data now has an additional column. (n_ratings)")


Books_ratings has 1748097 rows.
Books_ratings now has 1287419 rows. -460678 rows. 
Books_data now has an additional column. (n_ratings)


In [23]:
#purge books where n_ratings < 10 (redundant operation)
prevBooks = len(books_data_1['title'])
books_data_1 = books_data_1[books_data_1['n_ratings']>=10]
print(f"Books data has " + str(books_data_1.columns.size) + " columns with " + str(books_data_1.shape[0]) + " rows each." + " -" + str(prevBooks-len(books_data_1['title'])) + " rows.")

Books data has 6 columns with 27991 rows each. -0 rows.


In [24]:
#Remove ratings from users who have rated less than 20 books.
prevRatings = len(books_ratings_1['title'])
x = books_ratings_1.groupby('user_id').count()['score'] >= 20 # since this is what amazon requires before a user can get recommendations
considerable_users = x[x].index
books_ratings_1 = books_ratings_1[books_ratings_1['user_id'].isin(considerable_users)]

print(f"Books ratings has " + str(books_ratings_1.shape[0]) + " rows." + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
print(f"" + str(len(books_ratings_1['user_id'].value_counts())) + " users remain who have rated at least 20 books.")

Books ratings has 168980 rows. -1118439 rows.
3417 users remain who have rated at least 20 books.


In [25]:
avg_rating = books_ratings_1.groupby('title')["score"].mean().rename("avg_rating")
books_data_1 = books_data_1.merge(avg_rating, on='title', how='inner')
print(f"Books data now has an additional column. (avg_rating)")

Books data now has an additional column. (avg_rating)


In [26]:
#
def get_n_helpful(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        return int(denom)
    else:
        return x

def replace_fraction(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        if denom == '0':
            return 0
        else:
            return int(num) / int(denom)
    else:
        return x

books_ratings_1['helpfulness_count'] = books_ratings_1['helpfulness'].apply(get_n_helpful)
books_ratings_1['helpfulness_pct'] = books_ratings_1['helpfulness'].apply(replace_fraction)


In [27]:
import os
if not os.path.exists('clean_data'):
    os.makedirs('clean_data')

# save clean data
books_data_1.to_csv('clean_data/books_data_clean.csv', index=False)
books_ratings_1.to_csv('clean_data/books_rating_clean.csv', index=False)

prevBooks = len(books_data)
prevRatings = len(books_ratings)
newBooks = len(books_data_1)
newRatings = len(books_ratings_1)

print(f"First we had " + str(prevBooks) + " books and " + str(prevRatings) + " ratings.")
print(f"Now we have " + str(newBooks) + " books and " + str(newRatings) + " ratings. -" + str(round((prevBooks-newBooks)/prevBooks*100,2)) + "% books and -" + str(round((prevRatings-newRatings)/prevRatings*100,2)) + "% ratings.")

First we had 212404 books and 3000000 ratings.
Now we have 21294 books and 168980 ratings. -89.97% books and -94.37% ratings.


<h2>Code from here on down is not saved to the csv file.</h2>

In [28]:
import pandas as pd
books_data_1 = pd.read_csv('clean_data/books_data_clean.csv') #, nrows=1000)
print(f"Books data has " + str(books_data_1.columns.size) + " columns with " + str(books_data_1.shape[0]) + " rows each.")
books_ratings_1 = pd.read_csv('clean_data/books_rating_clean.csv') #, nrows=1000)
print(f"Books ratings has " + str(books_ratings_1.columns.size) + " columns with " + str(books_ratings_1.shape[0]) + " rows each.")

Books data has 7 columns with 21294 rows each.
Books ratings has 8 columns with 168980 rows each.


In [29]:
books_ratings_1['user_id'].value_counts()
#Looked at the ratings for the top 2 users and found nothing remarkable. The review text is cut off after a small number of words, so it is tough to see if they are automated reviews, as most of them more or less just describe the book rather than the reader's opinion of it.

user_id
AFVQZQ8PW0L       1963
A14OJS0VWMOSWO    1740
A1X8VZWTOG8IS6     765
A1K1JW1C5CUSUZ     677
A2F6N60Z96CAJI     557
                  ... 
ACK3TZEDR258N       20
AJN24TVEGZEI4       20
A1X9YMZQ2XXDCI      20
A31Y9DLKVASJQY      20
A2XZW1WV871DIO      20
Name: count, Length: 3417, dtype: int64

In [30]:
# extract all books rated by the top user by number of ratings, to check for duplicate reviews (spam)
books_ratings_1[books_ratings_1['user_id'] == 'AFVQZQ8PW0L']['summary'].value_counts()

summary
exciting thriller                          12
exciting romantic suspense                  8
deep character study                        6
A good read                                 6
exciting police procedural                  5
                                           ..
haunting classic war story                  1
fascinating historical mystery              1
engaging espionage romance                  1
exciting premise                            1
fantastic paranormal serial killer tale     1
Name: count, Length: 1779, dtype: int64

In [31]:
books_ratings_1[books_ratings_1['user_id'] == 'AFVQZQ8PW0L']['text'].loc[34]

'No female, whether they are of the blood or through marriage, survives THE SCARLETTI CURSE. They all violently die. One of the villagers Nicoletta knows first hand that her own mother was a victim of the curse. Still, she lives life to the fullest and uses her abilities to heal people. Her guardian helps hide the secret that Nicoletta has empathic talents by fostering the idea she is just the village healer to keep her safe from Giovanni Scarletti.However, the kind Nicoletta helps cure ailing lass, suffering from dining on a contaminated meal with her uncle Giovanni. He immediately realizes that Nicoletta is his life mate and will protect her from his enemies, but will his need to keep her safe end up making him follow the footsteps of his ancestors who loved so deep they killed for love?Though there is a political sidebar that adds suspense to the tale, this gothic story line belongs to the two lead characters. Nicoletta is the classic heroine willing to do the right thing even at th

In [32]:
books_data_1.description.value_counts()

description
A Simon & Schuster eBook. Simon & Schuster has a great book for every reader.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [33]:
books_data_1[books_data_1['description'] == 'Presents a collection of stories selected from magazines in the United States and Canada']

,title,description,authors,published_date,categories,n_ratings,avg_rating
10535,the best american short stories 2006,Presents a collection of stories selected from...,"['Ann Patchett', 'Katrina Kenison']",2006,['Fiction'],14,2.666667
11390,the best american short stories 2004,Presents a collection of stories selected from...,"['Michael Chabon', 'Katrina Kenison']",2005,['Fiction'],15,4.166667
17768,the best american short stories 2001,Presents a collection of stories selected from...,"['Garrison Keillor', 'Katrina Kenison']",1998,['Fiction'],11,4.800000
17801,the best american mystery stories 2001 cd,Presents a collection of stories selected from...,"['Barbara Kingsolver', 'Katrina Kenison']",2001,['Fiction'],11,2.666667


In [34]:
books_data_1[books_data_1['description'] == "The core concepts and technologies of Windows networking Networking can be a complex topic, especially for those new tothe field of IT. This focused, full-color book takes a uniqueapproach to teaching Windows networking to beginners by strippingdown a network to its bare basics, thereby making each topic clearand easy to understand. Focusing on the new Microsoft TechnologyAssociate (MTA) program, this book pares down to just theessentials, showing beginners how to gain a solid foundation forunderstanding networking concepts upon which more advanced topicsand technologies can be built. This straightforward guide begins each chapter by laying out alist of topics to be discussed, followed by a concise discussion ofthe core networking skills you need to have to gain a strong handleon the subject matter. Chapters conclude with review questions andsuggested labs so you can measure your level of understanding ofthe chapter's content. Serves as an ideal resource for gaining a solid understandingof fundamental networking concepts and skills Offers a straightforward and direct approach to networkingbasics and covers network management tools, TCP/IP, the nameresolution process, and network protocols and topologies Reviews all the topics you need to know for taking the MTA98-366 exam Provides an overview of networking components, discussesconnecting computers to a network, and looks at connecting networkswith routers If you're new to IT and interested in entering the IT workforce,then Microsoft Windows Networking Essentials is essentialreading."]

,title,description,authors,published_date,categories,n_ratings,avg_rating
10711,microsoftandreg office xp inside out,The core concepts and technologies of Windows ...,['Darril Gibson'],2011-05-02,['Computers'],13,4.333333


In [35]:
prevBooks = len(books_data_1['title'])
books_data_1.drop_duplicates(subset=['description','authors'], keep='first', inplace=True)
print(f"Books data has " + str(books_data_1.shape[0]) + " rows." + " -" + str(prevBooks-len(books_data_1['title'])) + " rows.")


Books data has 21294 rows. -0 rows.


In [36]:
popular_books = books_ratings_1['title'].value_counts()[0:20]

In [37]:
popular_books

title
harry potter and the sorcerers stone                              285
the great gatsby                                                  226
the catcher in the rye                                            220
to kill a mockingbird                                             214
1984                                                              196
the hobbitt or there and back again illustrated by the author     192
harry potter and the chamber of secrets                           190
fahrenheit 451                                                    190
night                                                             183
memoirs of a geisha                                               182
john adams                                                        182
five people you meet in heaven                                    180
animal farm                                                       177
harry potter and the prisoner of azkaban                          177
blink the powe

In [38]:
books_ratings_1[books_ratings_1['title'] == 'the hobbit']

,title,user_id,helpfulness,score,summary,text,helpfulness_count,helpfulness_pct
32988,the hobbit,A1D2C0WDCSHUWZ,2/3,5.0,"There, and back again","""The Hobbit or There and Back Again"" is the fi...",3,0.666667
32989,the hobbit,A172VRBP4BZXK6,3/58,1.0,The Hobbit,I really did not like this book. There wasn't ...,58,0.051724
32990,the hobbit,A35PU6DDT4SJXK,0/0,5.0,"Great Book, Great Prequel!","The hobbit, a great book in its own right, als...",0,0.000000
32991,the hobbit,A17BT73RVRQYSP,0/0,5.0,Review of the comic book version. Its worth yo...,[...]There are enough adjatives already said o...,0,0.000000
32992,the hobbit,A36WD8GXQ5QCJ7,0/1,5.0,An amazing novel,I just finished reading The Hobbit for the fir...,1,0.000000
32993,the hobbit,A1FY3V7A2XRDG0,6/7,5.0,The Hobbit -- 70th Anniversary,I've been a huge Tolkien fan since I was a kid...,7,0.857143
32994,the hobbit,AAEP8YFERQ8FC,6/7,5.0,Bilbo and the Thirteen Dwarves,"When I was in fourth or fifth grade, every aft...",7,0.857143
32995,the hobbit,A2ATWKOFJXRRR1,6/7,5.0,The first truly five star book,Masterpiece. A work of Art. A lasting piece do...,7,0.857143
32996,the hobbit,A1DA6E4FNRSAWN,30/41,3.0,about this edition,I mean no offence but I am not going to review...,41,0.731707
